## Package Import

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from ipywidgets import IntSlider, HBox, interactive_output
import ipywidgets as widgets
import os
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import skew, kurtosis
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar
import seaborn as sns
from functools import reduce
import talib
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from utils import *

***
### Notes
This notebook is aimed for the data restructure (pipeline) in order to be compatible with the future monthly return data classifiers. For more details on how we visualized and manipulated the broad assets data, please refer to the `initial_analysis.ipynb` notebook (i.e., section `Initial Visualizations`, `Aggregating Returns up to Weekly`). Now, we would like to use the weekly returns data file directly from the /data dir: `broad_assets_weekly_rets.xlsx`:
***

## Universe of Broad Asset Data Import

In [2]:
data_path = os.path.dirname(os.getcwd()) + '/data'
data_file = data_path + '/fpi_raw_data.xlsx'
data = pd.read_excel(data_file, sheet_name='Universe of broad assets', index_col=0, parse_dates=True)
broad_assets = data.copy()
broad_assets

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.0152,0.0757,-0.0023,0.0009,0.0422,0.0050,0.0373,0.0021,-0.0025,-0.0084,-0.0318
2000-08-01,0.0099,-0.0491,0.0079,0.0063,-0.0115,0.0404,-0.0292,0.0107,0.0096,0.0005,0.0146
2000-08-02,0.0007,-0.0178,0.0015,0.0037,0.0098,-0.0095,-0.0005,0.0010,0.0112,0.0012,0.0187
2000-08-03,0.0192,0.0761,0.0072,0.0031,-0.0031,-0.0210,-0.0307,-0.0095,0.0061,-0.0262,0.0072
2000-08-04,0.0142,-0.0030,0.0066,0.0062,0.0164,-0.0249,0.0129,0.0042,-0.0012,-0.0036,0.0326
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-30,0.0203,0.0258,-0.0144,-0.0047,0.0135,0.0194,0.0043,0.0039,0.0070,-0.0145,-0.0239
2024-09-03,-0.0423,-0.0631,0.0225,0.0078,-0.0618,0.0019,-0.0384,-0.0399,0.0028,-0.0086,-0.0392
2024-09-04,-0.0033,-0.0040,0.0250,0.0097,-0.0039,0.0043,-0.0051,-0.0007,-0.0092,0.0020,-0.0186


## Weekly Data Import

In [3]:
data_path = os.path.dirname(os.getcwd()) + '/data'
data_prefix = '/broad_assets_'
weekly_rets_file = data_path + data_prefix + 'weekly_rets.xlsx'
weekly_rets = pd.read_excel(weekly_rets_file, index_col=0)
weekly_rets

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2000-07-31,0.0152,0.0757,-0.0023,0.0009,0.0422,0.0050,0.0373,0.0021,-0.0025,-0.0084,-0.0318
2000-08-07,0.0679,0.0526,0.0159,0.0134,0.0365,-0.0237,-0.0196,-0.0116,0.0241,-0.0266,0.0346
2000-08-14,0.0157,0.0023,0.0152,0.0063,0.0171,0.0254,0.0294,0.0369,0.0085,0.0063,0.0481
2000-08-21,0.0095,0.0566,-0.0018,-0.0003,0.0068,-0.0264,0.0016,-0.0024,0.0077,0.0018,0.0384
2000-08-28,0.0184,0.0653,0.0019,0.0041,0.0379,-0.0353,0.0195,-0.0170,0.0019,-0.0095,0.0313
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.0607,0.0718,-0.0440,-0.0147,0.0210,0.0425,0.0431,0.0882,0.0093,0.0503,0.0583
2024-08-19,0.1006,0.1346,0.0199,-0.0004,0.1037,0.0321,0.0931,0.0771,-0.0238,0.0239,-0.0377
2024-08-26,0.0020,-0.0266,0.0028,0.0064,0.0450,0.0540,0.0290,-0.0180,-0.0195,0.0097,0.0332


*** 
## Equity Domestic / Global Data Import

In [4]:
def natural_sort_key(asset_name):
    """
    Helper function to generate sorting key for natural sorting.
    Extracts numerical part of asset name for correct order (e.g., 'Asset 1', 'Asset 10').
    """
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', asset_name)]

def split_date(df, base_name="equity_dom_period"):
    """
    Splits data into multiple datasets based on the availability of asset data.
    Assumes 'Date' is already set as the index, ensuring columns are ordered as Asset 1, Asset 2, etc.
    Uses manual natural sorting to ensure assets like Asset 10 come after Asset 9, not Asset 1.
    Each split period is renamed according to a specified base name.

    Args:
    df (pd.DataFrame): The input dataframe with 'Date' as index and asset columns.
    base_name (str): Base name for each period key.

    Returns:
    dict: A dictionary where keys are renamed period identifiers and values are dataframes for each period.
    """
    
    periods = {}
    current_assets = set()
    period_start = df.index[0]
    
    for i in range(len(df)):
        row_assets = df.iloc[i].dropna().index
        
        # If the available assets change, finalize the current period and start a new one
        if set(row_assets) != current_assets:
            if current_assets:  # Only add a period if it's not the first iteration
                period_data = df[(df.index >= period_start) & (df.index < df.index[i])]
                sorted_columns = sorted(current_assets, key=natural_sort_key)
                period_data = period_data[sorted_columns]
                period_key = f"{base_name}_{len(periods) + 1}"
                periods[period_key] = period_data
            period_start = df.index[i]
            current_assets.update(row_assets)
    
    sorted_columns = sorted(current_assets, key=natural_sort_key)
    period_data = df[df.index >= period_start][sorted_columns]
    period_key = f"{base_name}_{len(periods) + 1}"
    periods[period_key] = period_data
    
    return periods

def export_split_dataframes(dataframes, prefix="equity_dom", output_folder="data"):
    """
    Exports each dataframe in the given dictionary to separate Excel files, with a customizable prefix.
    
    Args:
    dataframes (dict): Dictionary of dataframes to export.
    prefix (str): Prefix to use in naming each file (e.g., 'equity_glb' or 'equity_dom').
    output_folder (str): The folder where files will be saved.
    """
    data_path = os.path.dirname(os.getcwd()) + '/data'
    
    for i, (period_name, period_df) in enumerate(dataframes.items(), start=1):
        file_name = f"{prefix}_period_{i}.xlsx"
        file_path = os.path.join(data_path, file_name)
        period_df.to_excel(file_path)

In [5]:
data_path = os.path.dirname(os.getcwd()) + '/data'
data_file = data_path + '/fpi_raw_data.xlsx'
equity_domestic = pd.read_excel(data_file, sheet_name='Equity factor baskets domestic', index_col=0, parse_dates=True)
equity_global = pd.read_excel(data_file, sheet_name='Equity factor baskets global', index_col=0, parse_dates=True)
split_date_domestic = split_date(equity_domestic, base_name = 'equity_dom_period')
split_date_global = split_date(equity_global, base_name = 'equity_glb_period')
export_split_dataframes(split_date_domestic, prefix="equity_dom")
export_split_dataframes(split_date_global, prefix="equity_glb")

## Monthly Data Import

In [6]:
monthly_dom_returns = apply_monthly_returns(split_date_domestic)
monthly_glb_returns = apply_monthly_returns(split_date_global)

monthly_dom_returns_period_1 = monthly_dom_returns["equity_dom_period_1"]
monthly_dom_returns_period_2 = monthly_dom_returns["equity_dom_period_2"]
monthly_dom_returns_period_3 = monthly_dom_returns["equity_dom_period_3"]
monthly_dom_returns_period_4 = monthly_dom_returns["equity_dom_period_4"]

monthly_glb_returns_period_1 = monthly_glb_returns["equity_glb_period_1"]
monthly_glb_returns_period_2 = monthly_glb_returns["equity_glb_period_2"]
monthly_glb_returns_period_3 = monthly_glb_returns["equity_glb_period_3"]
monthly_glb_returns_period_4 = monthly_glb_returns["equity_glb_period_4"]
monthly_glb_returns_period_5 = monthly_glb_returns["equity_glb_period_5"]
monthly_glb_returns_period_6 = monthly_glb_returns["equity_glb_period_6"]
monthly_glb_returns_period_7 = monthly_glb_returns["equity_glb_period_7"]
monthly_glb_returns_period_8 = monthly_glb_returns["equity_glb_period_8"]
monthly_glb_returns_period_9 = monthly_glb_returns["equity_glb_period_9"]
monthly_glb_returns_period_10 = monthly_glb_returns["equity_glb_period_10"]
monthly_glb_returns_period_11 = monthly_glb_returns["equity_glb_period_11"]
monthly_dom_returns_period_4

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,...,Asset 37,Asset 38,Asset 39,Asset 40,Asset 41,Asset 42,Asset 43,Asset 44,Asset 45,Asset 46
Date,,,,,,,,,,,,,,,,,,,,,
2016-06-20,0.0097,0.0133,0.0193,0.0102,-0.0004,0.0157,0.0116,0.0115,0.0193,0.0050,...,0.0101,0.0133,0.0162,0.0090,0.0111,-0.0001,-0.0058,-0.0043,-0.0107,-0.0006
2016-07-01,0.0375,0.0095,-0.0468,0.0010,0.0624,0.0070,-0.0155,0.0098,-0.0389,0.0070,...,0.0077,0.0129,-0.0031,-0.0299,0.0068,0.0581,0.0541,0.0180,0.0497,0.0031
2016-08-01,0.0811,0.0680,0.0595,0.0806,0.0133,0.1185,0.0571,0.0069,0.1224,0.0464,...,0.0026,0.0420,0.0499,0.1176,0.0686,0.0092,-0.0117,-0.0036,-0.0038,-0.0012
2016-09-01,0.0392,-0.0453,-0.0085,0.0190,-0.0589,-0.0280,0.0251,0.0417,-0.0728,-0.0020,...,0.0353,0.0460,-0.0058,0.0717,-0.0065,-0.0505,-0.0825,-0.0051,0.0018,-0.0015
2016-10-03,-0.0121,0.0047,-0.0311,0.0043,0.0057,-0.0025,0.0032,-0.0318,-0.0563,-0.0100,...,-0.0323,-0.0450,0.0169,0.0525,0.0049,-0.0348,0.0057,-0.0015,-0.0194,0.0007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,-0.0672,-0.0762,-0.0498,-0.0160,-0.0570,-0.0474,0.0096,-0.0149,-0.0978,-0.0203,...,-0.0728,-0.0724,-0.1059,-0.0831,-0.0480,-0.0287,0.0441,-0.0186,-0.0398,-0.0003
2024-06-03,0.0726,0.0105,0.0971,0.0548,0.0147,0.0757,0.0090,0.0565,-0.0070,-0.0625,...,0.0137,0.0037,0.0567,0.1348,0.0015,-0.0282,0.0741,0.0206,0.0378,0.0063
2024-07-01,-0.0249,-0.0829,-0.0115,0.0312,0.0068,0.0321,-0.0595,-0.0115,-0.1055,-0.0501,...,0.0166,0.0102,-0.0685,-0.0050,-0.0899,0.0122,-0.0617,-0.0019,-0.0153,0.0034


In [7]:
data_path = os.path.dirname(os.getcwd()) + '/data'
equity_dom_monthly_rets_file = data_path + '/equity_domestic_monthly_rets.xlsx'
equity_glb_monthly_rets_file = data_path + '/equity_global_monthly_rets.xlsx'
equity_domestic_monthly_rets = pd.read_excel(equity_dom_monthly_rets_file, index_col=0)
equity_global_monthly_rets = pd.read_excel(equity_glb_monthly_rets_file, index_col=0)
display(equity_domestic_monthly_rets.head(), equity_global_monthly_rets.head())

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,...,Asset 37,Asset 38,Asset 39,Asset 40,Asset 41,Asset 42,Asset 43,Asset 44,Asset 45,Asset 46
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-03,0.0000,0.0000,0.0000,-0.0338,0.0000,0.0000,0.0000,-0.0137,0.0000,0.0000,...,0.0115,0.0419,0.0675,0.1113,0.0419,0.0177,-0.0411,-0.0118,-0.0171,-0.0013
2001-02-01,0.0000,0.0000,0.0000,-0.0378,0.0000,0.0000,0.0000,0.0048,0.0000,0.0000,...,0.0166,0.0215,-0.0046,0.0319,0.0253,-0.0520,-0.0496,0.0120,0.0114,0.0100
2001-03-01,0.0000,0.0000,0.0000,0.0269,0.0000,0.0000,0.0000,0.0441,0.0000,0.0000,...,-0.0172,0.0704,0.0395,-0.1746,0.0087,0.0011,0.0999,0.0110,0.0135,0.0058
2001-04-02,0.0000,0.0000,0.0000,-0.0210,0.0000,0.0000,0.0000,0.0367,0.0000,0.0000,...,-0.0038,-0.0238,-0.0028,-0.1542,-0.0019,-0.0201,0.0181,0.0008,-0.0176,0.0081
2001-05-01,0.0000,0.0000,0.0000,0.0414,0.0000,0.0000,0.0000,0.0314,0.0000,0.0000,...,0.0053,0.1430,0.0607,0.2815,0.0649,0.0244,0.0337,-0.0122,-0.0239,0.0032


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,...,Asset 43,Asset 44,Asset 45,Asset 46,Asset 47,Asset 48,Asset 49,Asset 50,Asset 51,Asset 52
Date,,,,,,,,,,,,,,,,,,,,,
2003-04-14,0.0059,0.0035,0.0000,0.0069,0.0000,0.0000,0.0090,0.0042,0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0028,-0.0009,-0.0028,0.0000,0.0000,0.0207,0.0204
2003-05-01,0.1023,0.1299,0.0000,0.0811,0.0000,0.0000,0.0997,0.0606,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0328,0.0056,0.0169,0.0000,0.0000,0.0332,0.0638
2003-06-02,0.0422,0.3734,0.0000,0.1223,0.0000,0.0000,0.1478,0.2189,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0603,0.0047,0.0370,0.0000,0.0000,0.0593,0.0683
2003-07-01,0.1001,0.1734,0.0000,0.0544,0.0000,0.0000,0.0618,0.1381,0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0115,0.0035,-0.0025,0.0000,0.0000,0.0158,0.0260
2003-08-01,0.0754,0.1196,0.0000,-0.0382,0.0000,0.0000,0.0560,0.0665,0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0922,-0.0049,-0.0515,0.0000,0.0000,-0.0002,0.0366


***
## Indicators Rebuild (270 for each asset)

#### ROC

In [8]:
def calculate_roc(data, periods):
    """
    Calculate ROC for each specified period and concatenate them into a single DataFrame.
    
    :param data: DataFrame containing return data for one asset
    :param periods: List of periods for calculating ROC
    :return: DataFrame with ROC indicators for each specified period
    """
    roc_dfs = []
    for period in periods:
        roc_df = data.rolling(window=period).apply(lambda x: np.prod(1 + x) - 1, raw=False)
        roc_df.columns = [f'ROC{period}_{col}' for col in data.columns]
        roc_dfs.append(roc_df)

    return pd.concat(roc_dfs, axis=1)

### MACD

In [9]:
def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    """
    Calculate MACD for each asset.
    
    :param data: DataFrame containing return data for one asset
    :param short_period: Short-term EMA period (default 12)
    :param long_period: Long-term EMA period (default 26)
    :param signal_period: Signal line EMA period (default 9)
    :return: DataFrame with MACD line, Signal line, and MACD histogram
    """
    macd_df = pd.DataFrame(index=data.index)
    
    for col in data.columns:
        short_ema = data[col].ewm(span=short_period, adjust=False).mean()
        long_ema = data[col].ewm(span=long_period, adjust=False).mean()
        macd_line = short_ema - long_ema
        signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
        macd_histogram = macd_line - signal_line
        
        macd_df[f'MACD_Line_{col}'] = macd_line
        macd_df[f'Signal_Line_{col}'] = signal_line
        macd_df[f'MACD_Histogram_{col}'] = macd_histogram
    
    return macd_df

### RSI

In [10]:
def calculate_rsi(data, window=14):
    """
    Calculate the Relative Strength Index (RSI) for a pandas DataFrame or Series that contains
    daily return data.
    
    Parameters:
    data (pd.Series or pd.DataFrame): Daily returns.
    window (int): The period for calculating RSI, default is 14.
    
    Returns:
    pd.DataFrame: RSI and Stochastic RSI values.
    """
    # Positive and negative gains
    gain = np.where(data > 0, data, 0)
    loss = np.where(data < 0, -data, 0)
    
    if isinstance(data, pd.DataFrame):
        gain = pd.DataFrame(gain, index=data.index, columns=data.columns)
        loss = pd.DataFrame(loss, index=data.index, columns=data.columns)
    else:
        gain = pd.Series(gain, index=data.index)
        loss = pd.Series(loss, index=data.index)

    # Calculate the rolling average gain and loss
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()

    rs = avg_gain / avg_loss

    rsi = 100 - (100 / (1 + rs))
    rsi.columns = [f'RSI_{col}' for col in rsi.columns]

    # Calculate Stochastic RSI
    stoch_rsi = (rsi - rsi.rolling(window=window).min()) / (rsi.rolling(window=window).max() - rsi.rolling(window=window).min())
    stoch_rsi.columns = [f'Stoch_{col}' for col in stoch_rsi.columns]
    rsi_df = pd.concat([rsi, stoch_rsi], axis=1)

    return rsi_df.dropna()

### Bollinger Bands

In [11]:
def calculate_bollinger_bands(data, windows=[20, 90, 180, 240], num_std=2):
    """
    Compute Bollinger Bands for multiple rolling window sizes.

    Parameters:
    - data (pd.DataFrame): A DataFrame of daily percent return data.
    - windows (list of int): List of rolling window sizes for moving averages.
    - num_std (int): Number of standard deviations for the bands (default is 2).

    Returns:
    - pd.DataFrame: Data with Bollinger Bands (MA, Upper, Lower) for each specified window.
    """
    bollinger_dfs = []
    
    for window in windows:
        # Calculate moving average and rolling std deviation for each window
        rolling_mean = data.rolling(window=window).mean()
        rolling_std = data.rolling(window=window).std()
        
        # Calculate upper and lower Bollinger Bands
        upper_band = rolling_mean + (rolling_std * num_std)
        lower_band = rolling_mean - (rolling_std * num_std)
        
        # Rename columns to include the window size
        rolling_mean.columns = [f'SMA{window}_{col}' for col in data.columns]
        upper_band.columns = [f'Upper{window}_{col}' for col in data.columns]
        lower_band.columns = [f'Lower{window}_{col}' for col in data.columns]
        
        # Combine the bands and moving average into one DataFrame
        bollinger_df = pd.concat([rolling_mean, upper_band, lower_band], axis=1)
        bollinger_dfs.append(bollinger_df)
    
    return pd.concat(bollinger_dfs, axis=1).dropna()

### Hilbert Transform

In [12]:
def calc_hilbert(returns):
    data = (1 + returns).cumprod()
    ht_transform = pd.DataFrame()

    for asset in returns.columns:
        ht_transform['ht_dcperiod_' + asset] = talib.HT_DCPERIOD(data[asset])                   # Calculate Hilbert Transform - Dominant Cycle Period
        ht_transform['ht_dcphase_' + asset] = talib.HT_DCPHASE(data[asset])                     # Calculate Hilbert Transform - Dominant Cycle Phase
        ht_transform['inphase_' + asset], data['quadrature'] = talib.HT_PHASOR(data[asset])     # Calculate Hilbert Transform - Phasor Components
        ht_transform['sine_' + asset], data['leadsine'] = talib.HT_SINE(data[asset])            # Calculate Hilbert Transform - SineWave
        ht_transform['ht_trendmode_' + asset] = talib.HT_TRENDMODE(data[asset])                 # Calculate Hilbert Transform - Trend vs Cycle Mode

    return ht_transform.dropna()

### EWMA Volatility

In [13]:
def calc_ewma_volatility(excess_returns, asset_col, theta=0.94, initial_vol=0.2 / np.sqrt(252)):
    """
    Calculate EWMA volatility for a single asset.

    :param excess_returns: DataFrame containing returns for one asset
    :param asset_col: Name of the asset column
    :param theta: Smoothing parameter for EWMA (default 0.94)
    :param initial_vol: Initial volatility value (default 0.2 / sqrt(252))
    :return: DataFrame with EWMA volatility for the specified asset
    """
    var_t0 = initial_vol ** 2
    ewma_var = [var_t0]
    
    for d in excess_returns.index:
        new_ewma_var = ewma_var[-1] * theta + (excess_returns.loc[d, asset_col] ** 2) * (1 - theta)
        ewma_var.append(new_ewma_var)
    ewma_var.pop(0)  # Remove the initial var_t0
    ewma_vol = [np.sqrt(v) for v in ewma_var]
    
    return pd.DataFrame({f'EWMA_vol_{asset_col}': ewma_vol}, index=excess_returns.index)

### Rolling Volatility

In [14]:
def calculate_rolling_volatilities(data, windows=[20, 60, 126, 252, 504]):
    """
    Calculate rolling volatilities for multiple windows.

    :param data: DataFrame with daily returns for each asset
    :param windows: List of rolling window periods
    :return: DataFrame with rolling volatilities for each asset and window
    """
    rolling_vol_dfs = []
    for window in windows:
        rolling_vol_df = data.rolling(window=window).std()
        rolling_vol_df.columns = [f'RollingVol_{window}_{col}' for col in data.columns]
        rolling_vol_dfs.append(rolling_vol_df)
        
    return pd.concat(rolling_vol_dfs, axis=1).dropna()

### Rolling Skew

In [15]:
def calculate_rolling_skew(data, windows=[20, 60, 126, 252, 504]):
    """
    Calculate rolling skew for multiple windows.

    :param data: DataFrame with daily returns for each asset
    :param windows: List of rolling window periods
    :return: DataFrame with rolling skew for each asset and window
    """
    rolling_skew_dfs = []
    for window in windows:
        rolling_skew_df = broad_assets.rolling(window=window).skew()
        rolling_skew_df.columns = [f'RollingSkew_{window}_{col}' for col in data.columns]
        rolling_skew_dfs.append(rolling_skew_df)
        
    return pd.concat(rolling_skew_dfs, axis=1).dropna()

### Rolling Kurtosis

In [16]:
def calculate_rolling_kurt(data, windows=[20, 60, 126, 252, 504]):
    """
    Calculate rolling kurtosis for multiple windows.

    :param data: DataFrame with daily returns for each asset
    :param windows: List of rolling window periods
    :return: DataFrame with rolling kurtosis for each asset and window
    """
    rolling_kurt_dfs = []
    for window in windows:
        rolling_kurt_df = broad_assets.rolling(window=window).kurt()
        rolling_kurt_df.columns = [f'RollingKurt_{window}_{col}' for col in data.columns]
        rolling_kurt_dfs.append(rolling_kurt_df)
        
    return pd.concat(rolling_kurt_dfs, axis=1).dropna()

In [23]:
def apply_indicators(data, 
                     indicators, 
                     assets, 
                     reference_data, 
                     theta=0.94, 
                     initial_vol=0.2 / np.sqrt(252),
                     rolling_windows=[20, 60, 126, 252, 504],
                     bollinger_windows=[20, 90, 180, 240],
                     lags=[1, 2, 4, 6, 12]):
    """
    Apply indicators, rolling volatilities, expanding skew, and Hilbert Transform indicators for each asset.
    
    :param data: DataFrame with weekly or monthly returns for each asset
    :param indicators: Dictionary of indicator functions
    :param assets: List of asset columns to process
    :param reference_data: DataFrame of daily returns for calculations
    :param theta: Smoothing parameter for EWMA
    :param initial_vol: Initial volatility for EWMA calculation
    :param rolling_windows: List of windows for calculating rolling volatilities
    :param lags: List of lag periods to create lagged variables
    """

    # Calculate indicators (MACD, RSI, Bollinger Bands) using daily data in reference_data
    macd_df = calculate_macd(reference_data, short_period=12, long_period=26, signal_period=9)
    rsi_df = calculate_rsi(reference_data, window=14)
    bollinger_df = calculate_bollinger_bands(reference_data, windows=bollinger_windows)

    # Calculate rolling statistics using daily data
    rolling_vol_dfs = {window: reference_data.rolling(window=window).std() for window in rolling_windows}
    rolling_skew_dfs = {window: reference_data.rolling(window=window).skew() for window in rolling_windows}
    rolling_kurt_dfs = {window: reference_data.rolling(window=window).kurt() for window in rolling_windows}

    # Expanding skew/kurtosis using daily data
    expanding_skew_df = reference_data.expanding(min_periods=(252 * 2)).skew().dropna()
    expanding_kurt_df = reference_data.expanding(min_periods=(252 * 2)).kurt().dropna()

    # Hilbert Transform using daily data
    hilbert_df = calc_hilbert(reference_data)

    # Loop through each asset and calculate indicators
    asset_dataframes = {}
    for asset in assets:
        indicator_dfs = []

        # Get MACD, RSI, and Bollinger Bands indicators for the asset
        macd_asset_df = macd_df[[f'MACD_Line_{asset}', f'Signal_Line_{asset}', f'MACD_Histogram_{asset}']]
        rsi_asset_df = rsi_df[[f'RSI_{asset}', f'Stoch_RSI_{asset}']]
        bollinger_asset_df = bollinger_df[[f'SMA{window}_{asset}' for window in bollinger_windows] + 
                                          [f'Upper{window}_{asset}' for window in bollinger_windows] +
                                          [f'Lower{window}_{asset}' for window in bollinger_windows]]

        # Append indicators
        indicator_dfs.extend([macd_asset_df, rsi_asset_df, bollinger_asset_df])

        # Apply custom indicators from the indicators dictionary
        for indicator_name, indicator_func in indicators.items():
            indicator_dfs.append(indicator_func(data[[asset]]))

        # Calculate EWMA volatility for each asset
        ewma_vol_df = calc_ewma_volatility(reference_data, asset, theta=theta, initial_vol=initial_vol)
        indicator_dfs.append(ewma_vol_df)

        # Add rolling and expanding stats
        for window, df in rolling_vol_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingVol_{window}_{asset}'}))
        for window, df in rolling_skew_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingSkew_{window}_{asset}'}))
        for window, df in rolling_kurt_dfs.items():
            indicator_dfs.append(df[[asset]].rename(columns={asset: f'RollingKurt_{window}_{asset}'}))
        indicator_dfs.extend([expanding_skew_df[[asset]].rename(columns={asset: f'ExpandingSkew_{asset}'}),
                              expanding_kurt_df[[asset]].rename(columns={asset: f'ExpandingKurt_{asset}'})])

        # Add Hilbert Transform for each asset
        indicator_dfs.append(hilbert_df[[f'ht_dcperiod_{asset}', f'ht_dcphase_{asset}', f'inphase_{asset}', 
                                         f'sine_{asset}', f'ht_trendmode_{asset}']])

        # Combine indicators and apply lags
        combined_df = pd.concat(indicator_dfs, axis=1).dropna()
        lagged_features = [combined_df.shift(lag).add_suffix(f'_lag{lag}') for lag in lags]
        combined_df = pd.concat([combined_df] + lagged_features, axis=1).dropna()

        # Add one-hot encoding for the asset and binary target variable (optional)
        one_hot_df = pd.DataFrame(0, index=combined_df.index, columns=[f'Asset_{i+1}' for i in range(len(assets))])
        one_hot_df[f'Asset_{assets.index(asset) + 1}'] = 1
        combined_df = pd.concat([combined_df, one_hot_df], axis=1)

        combined_df.columns = [re.sub(f'_Asset \d+', '', col) if not col.startswith('Asset_') else col
                               for col in combined_df.columns]      

        asset_dataframes[asset] = combined_df
    
    return asset_dataframes

"""
def apply_indicators(data, 
                     indicators, 
                     assets, 
                     broad_asset_data, 
                     theta=0.94, 
                     initial_vol=0.2 / np.sqrt(252),
                     rolling_windows=[20, 60, 126, 252, 504],
                     bollinger_windows=[20, 90, 180, 240],
                     lags=[1, 2, 4, 6, 12]):
    """
    Apply indicators, rolling volatilities, expanding skew, and Hilbert Transform indicators for each asset.

    :param data: DataFrame with weekly returns for each asset
    :param indicators: Dictionary of indicator functions
    :param assets: List of asset columns to process
    :param broad_asset_data: DataFrame of broad asset returns for calculations
    :param theta: Smoothing parameter for EWMA
    :param initial_vol: Initial volatility for EWMA calculation
    :param rolling_windows: List of windows for calculating rolling volatilities
    :param lags: List of lag periods to create lagged variables
    Apply indicators, rolling volatilities, expanding skew, and Hilbert Transform indicators for each asset.
    Add one-hot encoding and binary target variable (1 if return > 0, otherwise 0) to each asset's final DataFrame.
    """
    # MACD, RSI, and Bollinger Bands for all assets using daily data
    macd_df = calculate_macd(broad_asset_data, short_period=12, long_period=26, signal_period=9)
    rsi_df = calculate_rsi(broad_asset_data, window=14)  # This includes both RSI and Stochastic RSI
    bollinger_df = calculate_bollinger_bands(broad_asset_data, windows=bollinger_windows)

    # Rolling volatilies/skews/kurtoses using daily data
    rolling_vol_dfs = {window: broad_asset_data.rolling(window=window).std() for window in rolling_windows}
    rolling_skew_dfs = {window: broad_asset_data.rolling(window=window).skew() for window in rolling_windows}
    rolling_kurt_dfs = {window: broad_asset_data.rolling(window=window).kurt() for window in rolling_windows}

    # Expanding skew/kurtosis using daily data
    expanding_skew_df = broad_asset_data.expanding(min_periods=(252 * 2)).skew().dropna()
    expanding_kurt_df = broad_asset_data.expanding(min_periods=(252 * 2)).kurt().dropna()
    
    # Hilbert Trasnform using daily data
    hilbert_df = calc_hilbert(broad_asset_data)

    asset_dataframes = {}
    for asset in assets:
        indicator_dfs = []

        # 1. MACD Columns for the Current Asset
        macd_asset_columns = [f'MACD_Line_{asset}', f'Signal_Line_{asset}', f'MACD_Histogram_{asset}']
        macd_asset_df = macd_df[macd_asset_columns]
        
        # 2. RSI and Stochastic RSI Columns for the Current Asset
        rsi_asset_column = f'RSI_{asset}'
        stoch_rsi_asset_column = f'Stoch_RSI_{asset}'
        rsi_asset_df = rsi_df[[rsi_asset_column, stoch_rsi_asset_column]]
        
        # 3. Bollinger Bands Columns for the Current Asset
        bollinger_asset_columns = [f'SMA{window}_{asset}' for window in bollinger_windows] + \
                                  [f'Upper{window}_{asset}' for window in bollinger_windows] + \
                                  [f'Lower{window}_{asset}' for window in bollinger_windows]
        bollinger_asset_df = bollinger_df[bollinger_asset_columns]

        # Append MACD, RSI, Stochastic RSI, and Bollinger Bands for the specific asset
        indicator_dfs.extend([macd_asset_df, rsi_asset_df, bollinger_asset_df])
        
        # Apply each indicator in the pipeline
        for indicator_name, indicator_func in indicators.items():
            indicator_df = indicator_func(data[[asset]])
            indicator_dfs.append(indicator_df)
        
        # 4. EWMA volatility for the current asset using broad_asset_data
        ewma_vol_df = calc_ewma_volatility(broad_asset_data, asset, theta=theta, initial_vol=initial_vol)
        indicator_dfs.append(ewma_vol_df)
        
        # 5. Directly select only the rolling and expanding volatility, skew, and kurtosis columns for the current asset
        for window, rolling_vol_df in rolling_vol_dfs.items():
            rolling_vol_asset = rolling_vol_df[[asset]].rename(columns={asset: f'RollingVol_{window}_{asset}'})
            indicator_dfs.append(rolling_vol_asset)      
        
        for window, rolling_skew_df in rolling_skew_dfs.items():
            rolling_skew_asset = rolling_skew_df[[asset]].rename(columns={asset: f'RollingSkew_{window}_{asset}'})
            indicator_dfs.append(rolling_skew_asset)

        for window, rolling_kurt_df in rolling_kurt_dfs.items():
            rolling_kurt_asset = rolling_kurt_df[[asset]].rename(columns={asset: f'RollingKurt_{window}_{asset}'})
            indicator_dfs.append(rolling_kurt_asset)

        expanding_skew_asset = expanding_skew_df[[asset]].rename(columns={asset: f'ExpandingSkew_{asset}'})
        expanding_kurt_asset = expanding_kurt_df[[asset]].rename(columns={asset: f'ExpandingKurt_{asset}'})
        indicator_dfs.extend([expanding_skew_asset, expanding_kurt_asset])

        # 6. Hilbert Transform columns for the current asset
        hilbert_asset_columns = [f'ht_dcperiod_{asset}', f'ht_dcphase_{asset}', f'inphase_{asset}', f'sine_{asset}', f'ht_trendmode_{asset}']
        hilbert_asset_df = hilbert_df[hilbert_asset_columns]
        indicator_dfs.append(hilbert_asset_df)

        combined_df = pd.concat(indicator_dfs, axis=1).dropna()

        # Lagged features for each original feature
        lagged_features = []
        for lag in lags:
            lagged_df = combined_df.shift(lag).add_suffix(f'_lag{lag}')
            lagged_features.append(lagged_df)
        
        combined_df = pd.concat([combined_df] + lagged_features, axis=1).dropna()

        # Add one-hot encoding for the current asset
        one_hot_df = pd.DataFrame(0, index=combined_df.index, columns=[f'Asset_{i+1}' for i in range(len(assets))])
        one_hot_df[f'Asset_{assets.index(asset) + 1}'] = 1  # Set 1 for the current asset column
        combined_df = pd.concat([combined_df, one_hot_df], axis=1)

        # Add the binary target variable (shifted weekly return for the asset)
        #shifted_target = weekly_rets[asset].shift(-1).reindex(combined_df.index)
        #combined_df['target'] = np.where(shifted_target > 0, 1, 0)  # Convert to binary target

        combined_df.columns = [re.sub(f'_{asset}$', '', col) if not col.startswith('Asset_') else col
                               for col in combined_df.columns]        

        asset_dataframes[asset] = combined_df#.dropna(subset=['target'])
    
    return asset_dataframes
"""

***
## Indicators for Broad Assets

In [24]:
roc_periods = [4, 6, 12, 26, 52]
asset_columns = [col for col in weekly_rets.columns if col.startswith('Asset')]

indicators = {
    'ROC': lambda df: calculate_roc(df, roc_periods)
}

asset_dataframes = apply_indicators(
    data=weekly_rets,             # Weekly returns data for each asset
    indicators=indicators,        # Dictionary of indicators
    assets=asset_columns,         # List of asset columns
    reference_data=broad_assets   # Daily return data for indicators calculation
)

asset_dataframes[asset_columns[0]]

,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_2,Asset_3,Asset_4,Asset_5,Asset_6,Asset_7,Asset_8,Asset_9,Asset_10,Asset_11
Date,,,,,,,,,,,,,,,,,,,,,
2002-10-28,0.0012,0.0045,-0.0033,69.8340,0.8991,0.0094,-0.0022,-0.0021,-0.0017,0.1054,...,0,0,0,0,0,0,0,0,0,0
2002-11-04,0.0024,0.0022,0.0003,58.9595,0.3182,0.0150,-0.0014,-0.0016,-0.0016,0.0955,...,0,0,0,0,0,0,0,0,0,0
2002-11-11,-0.0060,-0.0006,-0.0054,45.7714,0.0000,0.0044,-0.0022,-0.0023,-0.0018,0.0757,...,0,0,0,0,0,0,0,0,0,0
2002-11-18,-0.0007,-0.0006,-0.0001,57.1682,0.4351,0.0003,0.0000,-0.0023,-0.0019,0.0531,...,0,0,0,0,0,0,0,0,0,0
2002-11-25,0.0022,0.0013,0.0008,56.5647,0.5058,0.0050,0.0026,-0.0021,-0.0013,0.0577,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-12,0.0013,-0.0014,0.0027,38.7241,0.7106,-0.0052,0.0005,0.0018,0.0015,0.0487,...,0,0,0,0,0,0,0,0,0,0
2024-08-19,0.0054,0.0028,0.0025,59.3434,1.0000,0.0009,0.0018,0.0023,0.0018,0.0572,...,0,0,0,0,0,0,0,0,0,0
2024-08-26,0.0014,0.0025,-0.0011,80.5849,0.9299,0.0028,0.0024,0.0023,0.0019,0.0554,...,0,0,0,0,0,0,0,0,0,0


## Data Export for 11 Broad Assets

In [25]:
data_path = os.path.dirname(os.getcwd()) + '/data'
os.makedirs(data_path, exist_ok=True)

# Export each asset DataFrame to an Excel file
for i, (asset, df) in enumerate(asset_dataframes.items(), start=1):
    filename = os.path.join(data_path, f'broad_asset_{i}_features.xlsx')
    df.to_excel(filename, sheet_name=f'{asset}_features', index=True)
    print(f"Exported {asset} to {filename}")

Exported Asset 1 to /Users/a14542/Desktop/UChicago/24 AU/Flexible Plan Investments/fpi_project_lab_autumn2024/data/broad_asset_1_features.xlsx
Exported Asset 2 to /Users/a14542/Desktop/UChicago/24 AU/Flexible Plan Investments/fpi_project_lab_autumn2024/data/broad_asset_2_features.xlsx
Exported Asset 3 to /Users/a14542/Desktop/UChicago/24 AU/Flexible Plan Investments/fpi_project_lab_autumn2024/data/broad_asset_3_features.xlsx
Exported Asset 4 to /Users/a14542/Desktop/UChicago/24 AU/Flexible Plan Investments/fpi_project_lab_autumn2024/data/broad_asset_4_features.xlsx
Exported Asset 5 to /Users/a14542/Desktop/UChicago/24 AU/Flexible Plan Investments/fpi_project_lab_autumn2024/data/broad_asset_5_features.xlsx
Exported Asset 6 to /Users/a14542/Desktop/UChicago/24 AU/Flexible Plan Investments/fpi_project_lab_autumn2024/data/broad_asset_6_features.xlsx
Exported Asset 7 to /Users/a14542/Desktop/UChicago/24 AU/Flexible Plan Investments/fpi_project_lab_autumn2024/data/broad_asset_7_features.xlsx

***
## Indicators for Equity Domestic

In [26]:
def apply_indicators_to_all_periods(monthly_returns, split_data, roc_periods):
    """
    Applies the apply_indicators function to each period's monthly returns.

    Args:
    monthly_returns (dict): Dictionary of monthly return dataframes for each period.
    split_data (dict): Dictionary of daily return data for each period.
    roc_periods (list): List of periods to use for Rate of Change (ROC) calculation.

    Returns:
    dict: A dictionary with the same keys as monthly_returns, where each value is the
          result of apply_indicators for that period.
    """
    all_period_indicators = {}
    
    indicators = {
        'ROC': lambda df: calculate_roc(df, roc_periods)
    }
    
    for period_name, monthly_data in monthly_returns.items():
        asset_columns = [col for col in monthly_data.columns if col.startswith('Asset')]
        
        if not asset_columns:
            print(f"Warning: No asset columns found for period {period_name}. Skipping.")
            continue
        
        try:
            asset_dataframes = apply_indicators(
                data=monthly_data,
                indicators=indicators,
                assets=asset_columns,
                reference_data=split_data.get(period_name)
            )

            all_period_indicators[period_name] = asset_dataframes

        except KeyError as e:
            print(f"KeyError: {e} in period {period_name}. Please verify column names and data format.")
            continue
    
    return all_period_indicators


roc_periods = [1, 3, 6, 9, 12]
monthly_dom_indicators = apply_indicators_to_all_periods(monthly_dom_returns, split_date_domestic, roc_periods)
monthly_glb_indicators = apply_indicators_to_all_periods(monthly_glb_returns, split_date_global, roc_periods)

print('Asset Split Guidance of Equity Domestic Basket based on the Available Assets for Certain Periods:')
for period_name, indicator_data in monthly_dom_indicators.items():
    print(f"Available assets in {period_name}: {list(indicator_data.keys())}")

print('\nAsset Split Guidance of Equity Global Basket based on the Available Assets for Certain Periods:')
for period_name, indicator_data in monthly_glb_indicators.items():
    print(f"Available assets in {period_name}: {list(indicator_data.keys())}")

Asset Split Guidance of Equity Domestic Basket based on the Available Assets for Certain Periods:
Available assets in equity_dom_period_1: ['Asset 4', 'Asset 8', 'Asset 11', 'Asset 13', 'Asset 17', 'Asset 23', 'Asset 24', 'Asset 29', 'Asset 30', 'Asset 34', 'Asset 35', 'Asset 36', 'Asset 37', 'Asset 38', 'Asset 39', 'Asset 40', 'Asset 41', 'Asset 42', 'Asset 43', 'Asset 44', 'Asset 45', 'Asset 46']
Available assets in equity_dom_period_2: ['Asset 1', 'Asset 2', 'Asset 3', 'Asset 4', 'Asset 6', 'Asset 7', 'Asset 8', 'Asset 9', 'Asset 10', 'Asset 11', 'Asset 13', 'Asset 14', 'Asset 15', 'Asset 17', 'Asset 19', 'Asset 20', 'Asset 21', 'Asset 22', 'Asset 23', 'Asset 24', 'Asset 25', 'Asset 26', 'Asset 27', 'Asset 28', 'Asset 29', 'Asset 30', 'Asset 31', 'Asset 32', 'Asset 33', 'Asset 34', 'Asset 35', 'Asset 36', 'Asset 37', 'Asset 38', 'Asset 39', 'Asset 40', 'Asset 41', 'Asset 42', 'Asset 43', 'Asset 44', 'Asset 45', 'Asset 46']
Available assets in equity_dom_period_3: ['Asset 1', 'Asset 

In [27]:
target_period = "equity_dom_period_1"
data_dicts = [monthly_dom_indicators, monthly_glb_indicators]

for i, data_dict in enumerate(data_dicts, start=1):
    if target_period in data_dict:
        print(f"\nData from dictionary {i} for period '{target_period}':")
        
        for asset, data in data_dict[target_period].items():
            print(f"\nAsset: {asset}")
            display(data)
    else:
        print(f"No data found in dictionary {i} for period '{target_period}'.")


Data from dictionary 1 for period 'equity_dom_period_1':

Asset: Asset 4


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18,Asset_19,Asset_20,Asset_21,Asset_22
Date,,,,,,,,,,,,,,,,,,,,,
2004-02-02,-0.0013,-0.0017,0.0004,31.2700,0.0000,0.0007,0.0010,0.0012,0.0012,0.0254,...,0,0,0,0,0,0,0,0,0,0
2004-03-01,0.0012,-0.0001,0.0013,63.1342,0.7497,0.0024,0.0011,0.0012,0.0013,0.0243,...,0,0,0,0,0,0,0,0,0,0
2004-04-01,0.0023,0.0012,0.0011,56.8930,0.8402,-0.0000,0.0009,0.0012,0.0012,0.0263,...,0,0,0,0,0,0,0,0,0,0
2004-05-03,-0.0001,-0.0002,0.0001,64.2700,0.3558,0.0015,0.0011,0.0012,0.0013,0.0184,...,0,0,0,0,0,0,0,0,0,0
2004-06-01,0.0006,0.0001,0.0005,63.1161,0.9294,-0.0011,0.0001,0.0009,0.0009,0.0135,...,0,0,0,0,0,0,0,0,0,0
2004-07-01,-0.0000,0.0002,-0.0003,59.1560,0.4931,0.0009,0.0006,0.0007,0.0009,0.0156,...,0,0,0,0,0,0,0,0,0,0
2004-08-02,0.0026,0.0007,0.0019,59.5904,1.0000,0.0014,0.0012,0.0006,0.0008,0.0238,...,0,0,0,0,0,0,0,0,0,0
2004-09-01,0.0013,0.0004,0.0008,65.6788,1.0000,-0.0006,-0.0002,0.0004,0.0006,0.0215,...,0,0,0,0,0,0,0,0,0,0
2004-10-01,0.0023,0.0005,0.0018,60.4608,0.7179,0.0014,0.0007,0.0003,0.0007,0.0229,...,0,0,0,0,0,0,0,0,0,0



Asset: Asset 8


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18,Asset_19,Asset_20,Asset_21,Asset_22
Date,,,,,,,,,,,,,,,,,,,,,
2004-02-02,-0.0017,-0.0010,-0.0007,46.2449,0.0000,0.0004,0.0017,0.0012,0.0012,0.0156,...,0,0,0,0,0,0,0,0,0,0
2004-03-01,0.0025,0.0008,0.0016,88.1699,1.0000,0.0027,0.0021,0.0012,0.0015,0.0189,...,0,0,0,0,0,0,0,0,0,0
2004-04-01,0.0012,-0.0001,0.0013,52.3295,0.6981,0.0001,0.0018,0.0015,0.0013,0.0254,...,0,0,0,0,0,0,0,0,0,0
2004-05-03,0.0006,-0.0004,0.0009,55.1635,0.2853,0.0016,0.0011,0.0015,0.0013,0.0284,...,0,0,0,0,0,0,0,0,0,0
2004-06-01,0.0015,0.0004,0.0011,59.2241,1.0000,-0.0006,0.0005,0.0012,0.0010,0.0279,...,0,0,0,0,0,0,0,0,0,0
2004-07-01,-0.0015,-0.0002,-0.0013,60.6145,0.1222,0.0014,0.0009,0.0011,0.0012,0.0207,...,0,0,0,0,0,0,0,0,0,0
2004-08-02,0.0009,0.0001,0.0008,68.5110,1.0000,0.0010,0.0010,0.0013,0.0013,0.0126,...,0,0,0,0,0,0,0,0,0,0
2004-09-01,0.0010,0.0005,0.0005,78.6835,1.0000,0.0013,0.0009,0.0011,0.0013,0.0122,...,0,0,0,0,0,0,0,0,0,0
2004-10-01,0.0010,0.0004,0.0006,73.1127,1.0000,0.0017,0.0015,0.0010,0.0013,0.0170,...,0,0,0,0,0,0,0,0,0,0



Asset: Asset 11


...


Asset: Asset 45


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18,Asset_19,Asset_20,Asset_21,Asset_22
Date,,,,,,,,,,,,,,,,,,,,,
2004-02-02,-0.0006,-0.0005,-0.0001,50.0065,0.0000,0.0014,0.0006,0.0001,0.0004,0.0156,...,0,0,0,0,0,0,0,0,1,0
2004-03-01,0.0002,0.0001,0.0001,60.6265,0.4674,0.0013,0.0011,-0.0000,0.0005,0.0110,...,0,0,0,0,0,0,0,0,1,0
2004-04-01,-0.0010,-0.0009,-0.0001,40.3081,0.1288,0.0008,0.0008,0.0005,0.0005,0.0141,...,0,0,0,0,0,0,0,0,1,0
2004-05-03,0.0005,0.0000,0.0004,40.1704,1.0000,-0.0017,-0.0001,0.0005,-0.0001,0.0090,...,0,0,0,0,0,0,0,0,1,0
2004-06-01,0.0003,0.0009,-0.0006,56.8365,0.7718,-0.0006,-0.0006,0.0002,-0.0002,0.0120,...,0,0,0,0,0,0,0,0,1,0
2004-07-01,0.0009,0.0006,0.0003,67.1796,0.9983,0.0012,-0.0003,0.0003,0.0002,0.0151,...,0,0,0,0,0,0,0,0,1,0
2004-08-02,0.0002,-0.0004,0.0006,54.4173,0.2708,0.0002,-0.0006,0.0004,0.0005,0.0116,...,0,0,0,0,0,0,0,0,1,0
2004-09-01,0.0005,0.0002,0.0003,70.4852,0.5314,0.0018,0.0007,0.0004,0.0004,0.0116,...,0,0,0,0,0,0,0,0,1,0
2004-10-01,-0.0014,-0.0004,-0.0010,52.8107,0.0000,0.0007,0.0010,0.0002,0.0005,0.0120,...,0,0,0,0,0,0,0,0,1,0



Asset: Asset 46


,MACD_Line,Signal_Line,MACD_Histogram,RSI,Stoch_RSI,SMA20,SMA90,SMA180,SMA240,Upper20,...,Asset_13,Asset_14,Asset_15,Asset_16,Asset_17,Asset_18,Asset_19,Asset_20,Asset_21,Asset_22
Date,,,,,,,,,,,,,,,,,,,,,
2004-02-02,-0.0001,-0.0001,0.0000,44.8567,0.0000,0.0002,0.0002,0.0001,0.0001,0.0024,...,0,0,0,0,0,0,0,0,0,1
2004-03-01,-0.0000,0.0000,-0.0000,62.3451,0.4999,0.0003,0.0002,0.0001,0.0002,0.0020,...,0,0,0,0,0,0,0,0,0,1
2004-04-01,-0.0002,-0.0001,-0.0001,42.0438,0.0000,0.0001,0.0002,0.0001,0.0002,0.0018,...,0,0,0,0,0,0,0,0,0,1
2004-05-03,0.0001,-0.0000,0.0001,37.9333,1.0000,-0.0003,0.0000,0.0001,0.0001,0.0022,...,0,0,0,0,0,0,0,0,0,1
2004-06-01,0.0000,0.0001,-0.0001,62.5749,0.7099,-0.0001,-0.0001,0.0001,0.0001,0.0024,...,0,0,0,0,0,0,0,0,0,1
2004-07-01,0.0002,0.0001,0.0001,63.5198,1.0000,0.0001,-0.0001,0.0001,0.0001,0.0030,...,0,0,0,0,0,0,0,0,0,1
2004-08-02,0.0001,-0.0000,0.0001,53.6424,0.3566,0.0000,-0.0001,0.0001,0.0001,0.0017,...,0,0,0,0,0,0,0,0,0,1
2004-09-01,0.0001,-0.0000,0.0001,70.2076,0.6057,0.0003,0.0001,0.0001,0.0001,0.0025,...,0,0,0,0,0,0,0,0,0,1
2004-10-01,-0.0001,-0.0001,-0.0000,46.3878,0.0000,-0.0000,0.0001,0.0000,0.0001,0.0020,...,0,0,0,0,0,0,0,0,0,1


No data found in dictionary 2 for period 'equity_dom_period_1'.
